In [1]:
import requests
import json
import os
import sqlalchemy
import pandas as pd
import mdapi_functions as md
from IPython.display import display, Markdown
import markdown
from bokeh.plotting import figure, show
from bokeh.models import (Span, TabPanel, Tabs, ColumnDataSource, DataCube,
                          GroupingInfo, StringFormatter, SumAggregator,
                          TableColumn, HoverTool)
from bokeh.palettes import Category20c
from bokeh.transform import cumsum
from bokeh.io import output_notebook
from math import pi
from datetime import datetime
import datacite_api_functions as dcf
import numpy as np
import data_doc_helper as dh

from math import pi


In [2]:


output_notebook(hide_banner=True)

x = {
    'United States': 157,
    'United Kingdom': 93,
    'Japan': 89,
    'China': 63,
    'Germany': 44,
    'India': 42,
    'Italy': 40,
    'Australia': 35,
    'Brazil': 32,
    'France': 31,
    'Taiwan': 31,
    'Spain': 29,
}

data = pd.Series(x).reset_index(name='value').rename(columns={'index': 'country'})
data['angle'] = data['value']/data['value'].sum() * 2*pi
data['color'] = Category20c[len(x)]

p = figure(height=350, title="Pie Chart", toolbar_location=None,
           tools="hover", tooltips="@country: @value", x_range=(-0.5, 1.0))

p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='country', source=data)

p.axis.axis_label = None
p.axis.visible = False
p.grid.grid_line_color = None

show(p)

In [3]:
output_notebook(hide_banner=True)

df = md.get_freeze_profile()
df = df[df["demo_grp"].isin(["ethnicity"])]
df2 = df[["cat", "n"]]
df2["angle"] = df2["n"]/ df2["n"].sum() * 2*pi
df2['color'] = Category20c[df2.__len__()]
df2["tot"] = df2["n"].sum()
df2

p = figure(height=350, title="Pie Chart", toolbar_location=None,
           tools="hover", tooltips="@cat: @n / @tot", x_range=(-0.5, 1.0))

p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='cat', source=df2)

p.axis.axis_label = None
p.axis.visible = False
p.grid.grid_line_color = None
show(p)


In [4]:
output_notebook(hide_banner=True)

df = md.get_freeze_profile()
df = df[df["demo_grp"].isin(["sex"])]
df2 = df[["cat", "n"]]
df2["angle"] = df2["n"]/ df2["n"].sum() * 2*pi
df2['color'] = Category20c[df2.__len__()]
df2["tot"] = df2["n"].sum()
df2

p = figure(height=350, title="Pie Chart", toolbar_location=None,
           tools="hover", tooltips="@cat: @n / @tot", x_range=(-0.5, 1.0))

p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='cat', source=df2)

p.axis.axis_label = None
p.axis.visible = False
p.grid.grid_line_color = None
show(p)


In [5]:
output_notebook(hide_banner=True)

df = md.get_freeze_profile()
df = df[df["demo_grp"].isin(["year_of_birth"])]
df2 = df[["cat", "n"]]
df2["angle"] = df2["n"]/ df2["n"].sum() * 2*pi
df2['color'] = Category20c[df2.__len__()]
df2["tot"] = df2["n"].sum()
df2

p = figure(height=350, title="Pie Chart", toolbar_location=None,
           tools="hover", tooltips="@cat: @n / @tot", x_range=(-0.5, 1.0))

p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='cat', source=df2)

p.axis.axis_label = None
p.axis.visible = False
p.grid.grid_line_color = None
show(p)


In [6]:
from data_doc_helper import PlaceDataSet as PDS

In [7]:
ds = PDS("air_pollution_hh")

In [8]:
ds.info_table()

Dataset Descriptor,Dataset-specific Information
Name of Dataset in TRE,GEO_air_pollution
Citation (APA),DOI and Citation TBC
Download Citation,DOI and Citation Downloads TBC
Owner,"University of Leicester and St George's, University of London"
Temporal Coverage,01/2010 - 12/2019
Geographical Coverage,TBC
Participant Count,77594
Number of Variables,30
Number of Observations,116653
Key Link,TBC


In [22]:
def get_table_vars(source, table):
    url = ('https://metadata-api-4a09f2833a54.herokuapp.com/datasets/{{source_and_table_name}}?source={}&table_name={}'.format(source, table))
    r = requests.get(url, headers={'access_token': API_KEY})
    return pd.json_normalize(json.loads(r.text))


In [3]:
md.get_table_vars("geo", "air_pollution_hh")

,value,index,variable_label,value_label,variable_name
0,None,0,individual level id,None,cohortkey_e
1,None,1,Address start date,None,address_start_date
2,None,2,Address end date,None,address_end_date
3,None,3,Experian match code indicator,None,matchcode
4,None,4,Lower layer super output area (LSOA) encrypted,None,lsoa11_e
5,None,5,Middle super output area (LSOA) encrypted,None,msoa11_e
6,None,6,UK Region,None,region
7,None,7,Annual average nitrogen dioxide NO2 (ug/m3) fo...,None,no2_2010
8,None,8,Annual average nitrogen dioxide NO2 (ug/m3) fo...,None,no2_2011
9,None,9,Annual average nitrogen dioxide NO2 (ug/m3) fo...,None,no2_2012


In [6]:
API_KEY = os.environ['FASTAPI_KEY']

In [7]:
def get_place_var_info():
    url = ('https://metadata-api-4a09f2833a54.herokuapp.com/place-var-info/')
    r = requests.get(url, headers={'access_token': API_KEY})
    return pd.json_normalize(json.loads(r.text))

In [11]:
dfv = get_place_var_info()
dfv = dfv[dfv["dataset"] == "air_pollution_hh"][["variable", "owner", "category", "date_range"]]
dfv

,variable,owner,category,date_range
0,cohortkey_e,LPS,Geography,N/A
1,address_start_date,LPS,Geography,N/A
2,address_end_date,LPS,Geography,N/A
3,matchcode,University of Leicester,Geography,N/A
4,lsoa11_e,University of Leicester,Geography,2011
5,msoa11_e,University of Leicester,Geography,2011
6,region,University of Leicester,Geography,2011
7,no2_2010,University of Leicester,Pollution,2010
8,no2_2011,University of Leicester,Pollution,2011
9,no2_2012,University of Leicester,Pollution,2012


In [17]:
def get_place_dataset_info():
    url = ('https://metadata-api-4a09f2833a54.herokuapp.com/place-dataset-info/')
    r = requests.get(url, headers={'access_token': API_KEY})
    return pd.json_normalize(json.loads(r.text))

In [ ]:
df = get_place_dataset_info()
df = df[df["dataset"] == "air_pollution_hh"][["dataset", "geographical_coverage", "authors", "hyperlink"]]
df

,dataset,geographical_coverage,authors,hyperlink
0,air_pollution_hh,Great Britain,Calvin Jephcote & Kathryn Adams (University of...,https://guidebook.ukllc.ac.uk/docs/linked_geo_...
